# Training 'Flipper_06', a binary classifier to detect dolphin sounds

In this notebook we will use the database (spectrograms) we previously created in the ```database_creation_pamguard``` notebook to train a Ketos model. This first model is a binary classifier that takes 3 seconds long spectrograms and classify them into 2 classes: 1 (presence of dolphin click trains), or 0 (absence of click trains).

We will then use this model on acoustic recordings (.wav files of several minutes) as a detector, to spot trains of dolphin clicks.

The content of this notebook partly comes from the Ketos documentations. the comments and code were adpated to our current ClickLearn project. 


## Contents:

[1. Mouting drive, importing the libraries and setting the paths](#section1)  
[2. Creating the data feed and the model](#section2)  
[3. Creating and training the Neural Network](#section3)  
 

<a id="section1"></a>
##1. Mouting drive, importing the libraries and setting the paths




Mounting a google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Installing ketos

In [ ]:
!pip install ketos

     |████████████████████████████████| 181 kB 4.2 MB/s 
     |████████████████████████████████| 209 kB 34.4 MB/s 
  Created wheel for ketos: filename=ketos-2.4.0-py3-none-any.whl size=226286 sha256=b50a399580e1247e2bfe4ded44c77f798cadc0ed7ab87425f25a7bd58ab03d37
  Stored in directory: /root/.cache/pip/wheels/a9/74/ec/07b9dbb748e2c8bd8c2e71833a4ff12edd76f4e6b5864fdbcb
  Created wheel for datetime-glob: filename=datetime_glob-1.0.8-py3-none-any.whl size=7980 sha256=a32f85f1b04a5d720d01315e3fa7168fd1c5c2119d086ccfcd2ec63a7ffe2b5d
  Stored in directory: /root/.cache/pip/wheels/66/83/5f/fc21bcb17423bdbb629b5faea89d6307d6fc4b7f4976ded714
  Created wheel for lexery: filename=lexery-1.1.1-py3-none-any.whl size=4892 sha256=31e7ed8ced41ed4dab7b40dd72595a782ee6cb030c76da3db4f63ebed4165422
  Stored in directory: /root/.cache/pip/wheels/38/69/66/3916e07b71430a7628b43521720bc4a9d1a8d34f81843533f7
  Created wheel for version-parser: filename=version_parser-1.0.1-py3-none-any.whl size=4863 sha256=c20

In [ ]:
from ketos.data_handling import selection_table as sl
import ketos.data_handling.database_interface as dbi
from ketos.data_handling.parsing import load_audio_representation
from ketos.audio.spectrogram import MagSpectrogram
from ketos.audio.audio_loader import AudioFrameLoader
from ketos.data_handling.parsing import load_audio_representation
from ketos.neural_networks.resnet import ResNetInterface
from ketos.data_handling.data_feeding import BatchGenerator
import ketos.neural_networks.dev_utils.detection as det

from google.colab import files


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Identifying GPU device and listing:


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


<a id="section2"></a>
## 2. Creating the data feed and the model

opening Ketos database (spectrograms)

In [ ]:
db = dbi.open_file("gdrive/MyDrive/database_ketos_02.h5", 'r')

In [ ]:
db

File(filename=gdrive/MyDrive/database_ketos_02.h5, title='', mode='r', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/test (Group) ''
/test/data (Table(1112,), fletcher32, shuffle, zlib(1)) ''
  description := {
  "data": Float32Col(shape=(86, 8162), dflt=0.0, pos=0),
  "filename": StringCol(itemsize=100, shape=(), dflt=b'', pos=1),
  "id": UInt32Col(shape=(), dflt=0, pos=2),
  "label": UInt8Col(shape=(), dflt=0, pos=3),
  "offset": Float64Col(shape=(), dflt=0.0, pos=4)}
  byteorder := 'little'
  chunkshape := (1,)
/train (Group) ''
/train/data (Table(5455,), fletcher32, shuffle, zlib(1)) ''
  description := {
  "data": Float32Col(shape=(86, 8162), dflt=0.0, pos=0),
  "filename": StringCol(itemsize=100, shape=(), dflt=b'', pos=1),
  "id": UInt32Col(shape=(), dflt=0, pos=2),
  "label": UInt8Col(shape=(), dflt=0, pos=3),
  "offset": Float64Col(shape=(), dflt=0.0, pos=4)}
  byteorder := 'little

Attributing names

In [ ]:
# CAUTION: these names should not be changed when using the ketos build because they are unfortunately hard coded as train_data and val_data
train_data = dbi.open_table(db, "/train/data")
val_data = dbi.open_table(db, "/test/data")

In [ ]:
print(train_data.shape, val_data.shape)

(5455,) (1112,)


In [17]:
train_data[0][0].shape

(86, 8162)

Custom function for batch loaders:

In [ ]:
from skimage.transform import resize

In [18]:
def transform_batch_custom(X, Y):
    '''This function reshapes the spectrograms into squares so that the ketos training loop can handle them.'''
    temp = resize(X, (X.shape[0], 86, 86))
    x = temp.reshape(temp.shape[0],temp.shape[1],temp.shape[2],1)
    y = tf.one_hot(Y['label'], depth=2, axis=1).numpy()
    return x, y

train_generator = BatchGenerator(batch_size=128, data_table=train_data,
                                 output_transform_func=transform_batch_custom,
                                 shuffle=True, refresh_on_epoch_end=True)

val_generator = BatchGenerator(batch_size=128, data_table=val_data,
                                output_transform_func=transform_batch_custom,
                                shuffle=True, refresh_on_epoch_end=False)

recipe.json contains the model architecture

In [19]:
resnet = ResNetInterface.build_from_recipe_file("gdrive/MyDrive/recipe.json")

In [20]:
resnet.train_generator = train_generator
resnet.val_generator = val_generator

<a id="section3"></a>
## 3. Creating and training the Neural Network

### Training attempt #6

In [21]:
resnet.checkpoint_dir = "gdrive/MyDrive/content/models/checkpoints_flipper06"

In [22]:
#Original settings
resnet.early_stopping_monitor

{'baseline': 0.5,
 'decreasing': True,
 'delta': 0.1,
 'max_epochs': None,
 'metric': 'val_loss',
 'min_epochs': 5,
 'period': 10}

In [23]:
#change the early stopping rules to use val_CategoricalAccuracy instead of Val_loss
resnet.early_stopping_monitor = {'baseline': None,  #determines if a specific value is aimed at or not 
 'decreasing': False,        # determines the direction of what we consider as an improvement (increase or decrease) of the metric
 'delta': 0.01,              # determines what is an improvement between 2 epochs. Otherwise we consider it is not improving 
 'max_epochs': None,         # not documented...
 'metric': 'val_CategoricalAccuracy',   # the metric we are following for early stopping
 'min_epochs': 3,            # The #epoch from which the monitoring starts 
 'period': 3}                # number of epochs without improvement to stop training 

In [24]:
resnet.early_stopping_monitor

{'baseline': None,
 'decreasing': False,
 'delta': 0.01,
 'max_epochs': None,
 'metric': 'val_CategoricalAccuracy',
 'min_epochs': 3,
 'period': 3}

In [25]:
resnet.train_loop(n_epochs=14, verbose=True, checkpoint_freq = 1, early_stopping = True)


Epoch: 1 
train_loss: 0.12664449214935303
train_CategoricalAccuracy: 0.901 train_Precision: 0.936 train_Recall: 0.862 
val_loss: 0.5386421084403992
val_CategoricalAccuracy: 0.557 val_Precision: 1.000 val_Recall: 0.092 


Focus metric val_CategoricalAccuracy

Epoch: 2 
train_loss: 0.058505818247795105
train_CategoricalAccuracy: 0.954 train_Precision: 0.960 train_Recall: 0.948 
val_loss: 0.44273102283477783
val_CategoricalAccuracy: 0.535 val_Precision: 1.000 val_Recall: 0.048 


Focus metric val_CategoricalAccuracy

Epoch: 3 
train_loss: 0.037050385028123856
train_CategoricalAccuracy: 0.977 train_Precision: 0.969 train_Recall: 0.985 
val_loss: 0.4362502992153168
val_CategoricalAccuracy: 0.540 val_Precision: 1.000 val_Recall: 0.057 


Focus metric val_CategoricalAccuracy

Epoch: 4 
train_loss: 0.012789016589522362
train_CategoricalAccuracy: 0.998 train_Precision: 0.997 train_Recall: 0.999 
val_loss: 0.31589895486831665
val_CategoricalAccuracy: 0.693 val_Precision: 1.000 val_Recall: 0.372

{'checkpoint_name': 'cp-0010.ckpt'}

In [26]:
resnet.save_model('gdrive/MyDrive/content/models/Flipper_06_13epochs.kt',audio_repr_file='/content/gdrive/MyDrive/content/spec_config_spectrogram_01.json')

We finally get the index of the last epoch with improvement (early stopping), to be able to find the best model in `resnet`object

In [28]:
resnet.last_epoch_with_improvement

9